In [1]:
import sys
sys.path.append('../src_variable')

import network as nw
import torch
import generate_kvadraturen as gk
import bus
import road as rd
import traffic_lights as tl
import junction as jn
import numpy as np
import loading_json as load

In [2]:
# torch.autograd.set_detect_anomaly(True)

In [3]:
T = 1000
network = gk.generate_kvadraturen_minimal_junctions(T)
# for road in network.roads:
#     print(road.id)
# network = gk.generate_kvadraturen_wo_tl(T)

# ids_bw = ["lundsbro_bw", "elvegata_bw", "tollbod_6bw", "tollbod_5bw", "tollbod_4bw", 
#         "tollbod_3bw", "tollbod_2bw", "tollbod_1bw", "v_strand_7bw", "v_strand_6bw",
#             "v_strand_5bw", "v_strand_4bw", "v_strand_3bw", "v_strand_2bw", "v_strand_1bw"]            
            

# stops_bw = [("tollbod_6bw", 50), ("tollbod_3bw", 90), ("tollbod_1bw", 30), ("v_strand_3bw", 25)]
# # stops_bw = [("tollbod_6bw", 50), ("tollbod_3bw", 90)]

# times_bw = [40, 130, 190, 250]
# # times_bw = [10, 70]

# bus_bw = bus.Bus(ids_bw, stops_bw, times_bw, network, id = "2")

# ids_fw = ["v_strand_1fw", "v_strand_2fw", "v_strand_3fw", "h_w_2", "h_w_3",
#     "h_w_4", "festning_4fw", "festning_5fw", "festning_6fw", "festning_7fw",
#     "tollbod_4fw", "tollbod_5fw", "tollbod_6fw", "elvegata_fw", "lundsbro_fw"]

# stops_fw = [("h_w_3", 30), ("festning_5fw", 40), ("tollbod_4fw", 25), 
#             ("tollbod_6fw", 60)]

ids_bw = ["lundsbro_bw", "elvegata_bw", "tollbod_2bw", "tollbod_1bw", "v_strand_5bw", 
          "v_strand_4bw", "v_strand_3bw", "v_strand_2bw", "v_strand_1bw"]            
            

stops_bw = [("tollbod_2bw", 50), ("tollbod_1bw", 90), ("tollbod_1bw", 230), ("v_strand_1bw", 25)]

times_bw = [40, 130, 190, 250]

bus_bw = bus.Bus(ids_bw, stops_bw, times_bw, network, id = "2", start_time = 0.0)

ids_fw = ["v_strand_1fw", "h_w_2", "festning_3fw", "festning_4fw", "tollbod_2fw",
          "elvegata_fw", "lundsbro_fw"]

stops_fw = [("h_w_2", 130), ("festning_4fw", 40), ("tollbod_2fw", 25), 
            ("tollbod_2fw", 260)]
times_fw = [30, 110, 130, 230]


bus_fw = bus.Bus(ids_fw, stops_fw, times_fw, network, id = "1")


times_bw_2 = [240, 330, 390, 450]

times_fw_2 = [530, 610, 630, 830]

bus_bw_2 = bus.Bus(ids_bw, stops_bw, times_bw, network, id = "3", start_time = 200.0)
bus_fw_2 = bus.Bus(ids_fw, stops_fw, times_fw, network, id = "4", start_time = 500.0)


In [4]:
# Create actual network
roads = network.roads
junctions = network.junctions
T = network.T
# Don't store the densities 
# bus_network = nw.RoadNetwork(roads, junctions, T, [bus_fw, bus_bw], store_densities = False)
bus_network = nw.RoadNetwork(roads, junctions, T, [bus_fw, bus_bw, bus_fw_2, bus_bw_2], store_densities = True)

In [5]:
# # Checking network is correct:
# for road in network.roads:
#     print(road.id, road.b, road.left_pos, road.right_pos)

In [5]:
# with torch.autograd.set_detect_anomaly(True):
#     densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()
densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

Bus 2 reached bus stop 0 at time 19.539030075073242, should wait for 29.813297271728516 seconds
Bus 1 reached bus stop 0 at time 27.268585205078125, should wait for 29.612804412841797 seconds
Bus 1 reached bus stop 1 at time 87.25641632080078, should wait for 29.49942398071289 seconds
Bus 2 reached bus stop 1 at time 91.44937133789062, should wait for 38.13539505004883 seconds
Bus 2 reached bus stop 2 at time 146.64942932128906, should wait for 43.08590316772461 seconds
Bus 1 reached bus stop 2 at time 188.59906005859375, should wait for 29.444435119628906 seconds
Bus 3 reached bus stop 0 at time 220.09646606445312, should wait for 29.510475158691406 seconds
Bus 1 reached bus stop 3 at time 247.39987182617188, should wait for 29.963897705078125 seconds
Bus 3 reached bus stop 1 at time 292.3366394042969, should wait for 29.998613357543945 seconds
Bus 3 reached bus stop 2 at time 339.52886962890625, should wait for 29.606388092041016 seconds
Bus 2 reached bus stop 3 at time 366.779754638

In [7]:
# bus_lengths

In [6]:
densities = load.convert_from_tensor(densities)
queues = load.convert_from_tensor(queues)
bus_lengths = load.convert_from_tensor(bus_lengths)

In [8]:
sys.getsizeof(bus_delays)

232

In [7]:
bus_delays_dict = {0: [float(l.detach()) for l in bus_delays[0]],
                   1: [float(l.detach()) for l in bus_delays[1]]}

In [8]:
# Write above dictionaries to json file
import json
with open("kvadraturen_results_minimal_1000_variable.json", 'w') as fd:
    fd.write(json.dumps([densities, queues, bus_lengths, bus_delays_dict]))

In [9]:
import json
print("Loading results...")
f = open("kvadraturen_results_minimal_1000.json")
data = json.load(f)
f.close()
densities = data[0]
queues = data[1]
bus_lengths = data[2]
bus_delays = data[3]

Loading results...


In [10]:
import openGLUtilsGif as opgl
opgl.draw_busses_w_densities(bus_network, [bus_fw, bus_bw, bus_fw_2, bus_bw_2], bus_lengths,
                                    densities, output_name="test_minimal_1000_variable.gif")

Maximum and minimum x points: 7, -1
Maximum and minimum y points: 0, 8.9
Window created
End of simulation reached!
Saving GIF as: test_minimal_1000_variable.gif


: 

In [20]:
out_dens = None
in_dens = None
for i, road in enumerate(bus_network.roads):
    if road.id == "festning_3bw":
        out_dens = densities[i]
    elif road.id == "festning_4bw":
        in_dens = densities[i]

In [24]:
times = out_dens.keys()
for t in times:
    print(f"t={t}: {out_dens[t]}")

t=0.0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=0.5999999642372131: [0.08999999612569809, 0.08999999612569809, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=1.1999999284744263: [0.09702927619218826, 0.09702927619218826, 0.04372436925768852, 0.04020165652036667, 0.0, 0.0, 0.0, 0.0, 0.0]
t=1.7999999523162842: [0.09905938804149628, 0.09905938804149628, 0.07037248462438583, 0.06369003653526306, 0.02085283398628235, 0.0172907505184412, 0.0, 0.0, 0.0]
t=2.3999998569488525: [0.09971043467521667, 0.09971043467521667, 0.08475338667631149, 0.0799742341041565, 0.044029124081134796, 0.03632432594895363, 0.010183526203036308, 0.010131673887372017, 0.010131673887372017]
t=2.999999761581421: [0.09991445392370224, 0.09991445392370224, 0.09226521104574203, 0.08925274014472961, 0.06315527111291885, 0.05633240193128586, 0.0262756384909153, 0.035847216844558716, 0.035847216844558716]
t=3.5999996662139893: [0.09997563064098358, 0.09997563064098358, 0.09610311686992645, 0.09452551603317261, 0.0768801271915435

In [25]:
times = in_dens.keys()
for t in times:
    print(f"t={t}: {in_dens[t]}")

t=0.0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=0.5999999642372131: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=1.1999999284744263: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=1.7999999523162842: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=2.3999998569488525: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=2.999999761581421: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=3.5999996662139893: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t=4.199999809265137: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [32]:
for j in bus_network.junctions:
    j_roads = j.roads
    for road in j_roads:
        if road.id == "festning_3bw":
            print(f"Junction with the roads {[r.id for r in j_roads]}")
            # print(f"Entering {entering}")

Junction with the roads ['h_w_2', 'festning_2fw', 'festning_3fw', 'festning_2bw', 'festning_3bw']


In [33]:
for i, j in enumerate(bus_network.junctions):
    j_roads = j.roads
    print(f"Junction {i+1} has the roads {[r.id for r in j_roads]}")

Junction 1 has the roads ['v_strand_1fw', 'v_strand_2fw', 'v_strand_1bw', 'v_strand_2bw', 'h_w_1', 'h_w_2']
Junction 2 has the roads ['v_strand_2fw', 'v_strand_3fw', 'v_strand_2bw', 'v_strand_3bw']
Junction 3 has the roads ['v_strand_3fw', 'v_strand_4fw', 'v_strand_3bw', 'v_strand_4bw']
Junction 4 has the roads ['v_strand_4fw', 'v_strand_5fw', 'v_strand_4bw', 'v_strand_5bw']
Junction 5 has the roads ['v_strand_5fw', 'v_strand_6fw', 'v_strand_5bw', 'v_strand_6bw', 'tollbod_1bw']
Junction 6 has the roads ['v_strand_6fw', 'dronning_1fw', 'v_strand_6bw', 'dronning_1bw']
Junction 7 has the roads ['h_w_2', 'festning_2fw', 'festning_3fw', 'festning_2bw', 'festning_3bw']
Junction 8 has the roads ['tollbod_1bw', 'festning_4fw', 'festning_5fw', 'festning_4bw', 'festning_5bw', 'tollbod_2fw', 'tollbod_2bw']
Junction 9 has the roads ['tollbod_2fw', 'elvegata_fw', 'tollbod_2bw', 'elvegata_bw']
Junction 10 has the roads ['dronning_1fw', 'dronning_2fw', 'dronning_1bw', 'dronning_2bw']
Junction 11 has 

In [31]:
print(len(bus_network.junctions))


14


In [10]:
# Write above dictionaries to json file
import json
with open("kvadraturen_results_minimal.json", 'w') as fd:
    fd.write(json.dumps([densities, queues, bus_lengths, bus_delays_dict]))

In [16]:
f = open("kvadraturen_results.json")
data = json.load(f)
f.close()

In [8]:
objective = torch.tensor(0.0)
for i in range(len(bus_delays)):
    for delay in bus_delays[i]:
        objective += delay

In [10]:
objective.backward()

In [11]:
import openGLUtilsGif as opgl

In [12]:
# print(bus_delays)
opgl.draw_busses_timed(bus_network, [bus_fw, bus_bw], bus_lengths, 0.1)

Window created
End of simulation reached!
Saving GIF as: bus_animation.gif


: 

In [7]:
# Cloning just to be safe
# objective = bus_delays[0][0].clone() #+ bus_delays[1][0].clone()
# for i in range(len(bus_delays)):
#     for j in range(len(bus_delays[i])):
#         if i == 0 and j == 0:
#             continue
#         objective = objective + bus_delays[i][j].clone()
# objective = bus_delays[0][0] + bus_delays[0][1] + bus_delays[0][2]
objective = torch.tensor(0.0)

# Error with stop number 3 and 4 for fw bus and stop 4 for bw bus

# objective = bus_delays[0][0]  
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        # if j == 2:
        #     continue
        objective = objective + bus_delays[i][j]

In [8]:
print(objective)

tensor(250.7767, grad_fn=<AddBackward0>)


In [9]:
# objective.backward(retain_graph=True)
objective.backward(retain_graph=True)

In [10]:
params = [v for road in bus_network.roads for v in road.Vmax] +\
        [t for junction in bus_network.junctions for traffic_light in junction.trafficlights for t in traffic_light.cycle] +\
        [t for junction in bus_network.junctions for traffic_light in junction.coupled_trafficlights for t in traffic_light.cycle]

In [11]:
print("Road Vmax gradients:")
for road in bus_network.roads:
    for v in road.Vmax:
        print(v.grad)

Road Vmax gradients:
tensor(-0.2147)
tensor(5.1381e-09)
tensor(1.5925e-07)
tensor(2.1815e-05)
tensor(2.4246e-05)
tensor(-7.8203e-05)
tensor(7.5396e-05)
tensor(7.5433e-05)
tensor(-7.1640e-11)
tensor(-2.9962e-11)
tensor(-0.1284)
tensor(-0.0035)
tensor(-4.7124e-05)
tensor(-0.0002)
tensor(0.0002)
tensor(-8.8593e-06)
tensor(0.0002)
tensor(2.1430e-10)
tensor(-2.1953e-09)
tensor(0.)
tensor(-1.8762)
tensor(-1.4539)
tensor(-0.8621)
tensor(-0.0010)
tensor(-0.0007)
tensor(-0.0007)
tensor(-0.0008)
tensor(-0.0006)
tensor(-0.0006)
tensor(-4.5511e-07)
tensor(-0.0003)
tensor(0.0001)
tensor(-0.0016)
tensor(0.0001)
tensor(-3.3369e-06)
tensor(-1.2600e-05)
tensor(-9.5367e-05)
tensor(3.3264e-05)
tensor(2.0803e-06)
tensor(-0.0314)
tensor(-0.1452)
tensor(-0.0166)
tensor(0.0005)
tensor(-3.4343e-07)
tensor(-6.4915e-07)
tensor(-0.0007)
tensor(-2.7000e-06)
tensor(0.0174)
tensor(-0.0001)
tensor(0.0040)
tensor(-0.0435)
tensor(-0.0001)
tensor(-8.4380e-05)
tensor(-0.0016)
tensor(-0.0007)
tensor(-0.0306)
tensor(-0.01

In [12]:
for junction in bus_network.junctions:
    for traffic_light in junction.trafficlights:
        for t in traffic_light.cycle:
            print(t.grad)

    for traffic_light in junction.coupled_trafficlights:
        for t in traffic_light.cycle:
            print(t.grad)

tensor(2.7797e-06)
tensor(3.3811e-06)
tensor(-0.0002)
tensor(3.9738e-05)
tensor(0.0001)
tensor(9.1615e-05)
tensor(0.0001)
tensor(-3.3951e-05)
tensor(-0.0004)
tensor(8.7675e-10)
tensor(0.0001)
tensor(-8.0652e-09)
tensor(0.0005)
tensor(7.1534e-09)
tensor(-0.0001)
tensor(3.6390e-26)
tensor(1.6812e-05)
tensor(-2.9619e-06)
tensor(-0.0005)
tensor(9.5887e-05)
tensor(0.0004)
tensor(4.2679e-24)
tensor(7.4302e-05)
tensor(7.4316e-05)
tensor(0.0005)
tensor(0.0006)


In [13]:
for j in bus_network.junctions:
    print(j.get_activation(195.0, "tollbod_4fw", "tollbod_5fw"))
    # print(j.get_activation(195.0, "festning_7fw", "tollbod_4fw"))

    

(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(True, 1.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
